In [ ]:
from ctypes import cdll, c_size_t, c_double, POINTER, Structure, byref
from numpy.ctypeslib import as_ctypes
import numpy as np

my_lib = cdll.LoadLibrary("/path/to/array_recasting.so")

class CLikeArray(Structure):
    _fields_ = [("num_vecs", c_size_t), ("x", POINTER(c_double))]

    def __init__(self, num_vecs: int, x: np.array):
        self.num_vecs = num_vecs
        self._x = x
        self.x = as_ctypes(self._x.reshape([-1]))

In [ ]:
add = my_lib.array_sum
add.restype = None
add.argtypes = [POINTER(CLikeArray), POINTER(CLikeArray), POINTER(CLikeArray)]

In [ ]:
array_length = 20;
input_array_x = np.linspace(np.array([-0.5, 0.0, 0.1]), np.array([0.5, 10.0, 0.3]), array_length)
input_array_y = np.linspace(np.array([-0.5, 0.0, 0.1]), np.array([0.5, -10.0, -0.3]), array_length)
output_array = np.zeros_like(input_array_x)

in_x = CLikeArray(array_length, input_array_x)
in_y = CLikeArray(array_length, input_array_y)
out = CLikeArray(array_length, output_array)

In [ ]:
add(byref(in_x), byref(in_y), byref(out))

In [ ]:
out._x

In [ ]:
in_x._x + in_y._x

In [ ]:
#ifndef CXX_LIBRARY_TESTS_ARRAY_RECASTING_H
#define CXX_LIBRARY_TESTS_ARRAY_RECASTING_H

#include <vector>
#include <array>

extern "C" {
struct CLikeArray {
    std::size_t num_vecs;
    double *x;
};
}

class VectorStorer {
public:
    std::size_t num_vecs;
    std::vector <std::array<double, 3>> x;
    explicit VectorStorer(std::size_t n);
    explicit VectorStorer(CLikeArray &c_like_array);
    void copy_into_array(CLikeArray &c_like_array);
    static VectorStorer from_array(CLikeArray &c_like_array);
};

extern "C" {
void vec_store_sum(VectorStorer &in_x, VectorStorer &in_y, VectorStorer &out);
void array_sum(CLikeArray &x_in, CLikeArray &y_in, CLikeArray &out);
}

#endif //CXX_LIBRARY_TESTS_ARRAY_RECASTING_H


In [ ]:
#include "array_recasting.h"

VectorStorer::VectorStorer(std::size_t n) {
    num_vecs = n;
    for (int i = 0; i < num_vecs; ++i) {
        x.push_back({0.0, 0.0, 0.0});
    }
}

VectorStorer::VectorStorer(CLikeArray &c_like_array) {
    num_vecs = c_like_array.num_vecs;
    x = std::vector<std::array<double, 3 >>(reinterpret_cast<std::array<double, 3> *>(c_like_array.x),
                                            reinterpret_cast<std::array<double, 3> *>(c_like_array.x) + num_vecs);
}

void VectorStorer::copy_into_array(CLikeArray &c_like_array) {
    for (std::size_t i = 0; i < num_vecs; ++i) {
        for (std::size_t j = 0; j < 3; ++j) {
            c_like_array.x[3 * i + j] = x[i][j];
        }
    }
}

VectorStorer VectorStorer::from_array(CLikeArray &c_like_array) {
    auto vec_store = VectorStorer(c_like_array.num_vecs);
    for (std::size_t i = 0; i < c_like_array.num_vecs; ++i) {
        for (std::size_t j = 0; j < 3; ++j) {
            vec_store.x[i][j] = c_like_array.x[3 * i + j];
        }
    }
    return vec_store;
}

void vec_store_sum(VectorStorer &in_x, VectorStorer &in_y, VectorStorer &out) {
    for (std::size_t i = 0; i < in_x.num_vecs; ++i) {
        for (std::size_t j = 0; j < 3; ++j) {
            out.x[i][j] = in_x.x[i][j] + in_y.x[i][j];
        }
    }
}

void array_sum(CLikeArray &x_in, CLikeArray &y_in, CLikeArray &out) {
    auto u = VectorStorer(x_in);
    auto v = VectorStorer(y_in);
    auto w = VectorStorer(out);
    vec_store_sum(u, v, w);
    w.copy_into_array(out);
}